In [1]:
!pip install google-cloud-bigquery-storage openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 5.4 MB/s eta 0:00:0000:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.


In [ ]:
# Import Packages
from google.cloud import bigquery
from google.api_core.exceptions import NotFound
from datetime import datetime, timedelta
from openai import OpenAI
import pandas as pd
import numpy as np
import requests
import os
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
now = datetime.now()
year = now.year
month = now.strftime("%b").lower()  # jan, feb, mar
table_suffix = f"{year}_{month}"

In [3]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import textwrap

In [ ]:
# Initialize BigQuery client
client = bigquery.Client(project='data-storage-485106')

In [ ]:
# Define Table ID
table_id = f"data-storage-485106.food.market_prices_{table_suffix}"

# Define SQL Query to Retrieve Open Weather Data from Google Cloud BigQuery
sql = (f"""
        SELECT *
        FROM `{table_id}`
        WHERE commodity IN ("Dry Maize")
        ORDER BY date DESC
       """)
    
# Run SQL Query
data = client.query(sql).to_dataframe()

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29854 entries, 0 to 29853
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   commodity       29854 non-null  object 
 1   classification  29854 non-null  object 
 2   market          29854 non-null  object 
 3   wholesale       28076 non-null  float64
 4   retail          21761 non-null  float64
 5   supply_volume   17919 non-null  float64
 6   county          29768 non-null  object 
 7   date            29854 non-null  dbdate 
dtypes: dbdate(1), float64(3), object(4)
memory usage: 1.8+ MB


In [7]:
data.head()

,commodity,classification,market,wholesale,retail,supply_volume,county,date
0,Dry Maize,Mixed-Traditional,Kitale Municipality Market,45.00,55.0,300.0,Trans-Nzoia,2025-10-14
1,Dry Maize,Mixed-Traditional,Nkubu,44.44,50.0,630.0,Meru,2025-10-14
2,Dry Maize,White Maize,Gikomba,42.22,60.0,NaN,Nairobi,2025-10-14
3,Dry Maize,White Maize,Kangemi Market,50.00,NaN,NaN,Nairobi,2025-10-14
4,Dry Maize,White Maize,Kitale Municipality Market,50.00,60.0,300.0,Trans-Nzoia,2025-10-14


##### **1. National Price Change**

In [36]:
# Ensure date column is datetime
data["date"] = pd.to_datetime(data["date"])

# Assign each record to a week (starting Monday)
data["week"] = data["date"].dt.to_period("W-MON")

# Compute average retail price per week
national_weekly = (
    data.groupby("week", as_index=False)
        .agg(avg_retail_price=("retail", "mean"))
        .sort_values("week", ascending=True)
).round(2)

# Calculate week-over-week % change
national_weekly["pct_change"] = (
    national_weekly["avg_retail_price"].pct_change() * 100
).round(2)

# Convert week period to start and end dates for readability
national_weekly["week_start"] = national_weekly["week"].apply(lambda x: x.start_time.date())
national_weekly["week_end"] = national_weekly["week"].apply(lambda x: x.end_time.date())

# Get today's date (Nairobi timezone)
today = pd.Timestamp.now(tz="Africa/Nairobi").normalize().date()

# Label the most relevant week(s)
def label_week(row):
    if row["week_start"] <= today <= row["week_end"]:
        return "this week"
    elif row["week_end"] < today:
        return "last week"
    else:
        return "upcoming week"

national_weekly["week_label"] = national_weekly.apply(label_week, axis=1)

# Keep only the most recent two weeks for reporting
national_weekly = national_weekly.sort_values("week_start").iloc[-2:]

print("Weekly National Price Change\n")
national_weekly

Weekly National Price Change



,week,avg_retail_price,pct_change,week_start,week_end,week_label
420,2025-10-07/2025-10-13,57.76,-3.01,2025-10-07,2025-10-13,last week
421,2025-10-14/2025-10-20,62.14,7.58,2025-10-14,2025-10-20,this week


##### **2. Weekly County Price Change**

In [27]:
# --- Get start and end of this week ---
today = pd.Timestamp.today()
start_of_week = today - pd.to_timedelta(today.weekday(), unit="D")   # Monday
end_of_week = start_of_week + pd.Timedelta(days=6)                   # Sunday

# --- Define last week's range ---
last_week_start = start_of_week - pd.Timedelta(days=7)
last_week_end = end_of_week - pd.Timedelta(days=7)

# --- Filter both weeks ---
this_week = data[(data["date"] >= start_of_week) & (data["date"] <= end_of_week)]
last_week = data[(data["date"] >= last_week_start) & (data["date"] <= last_week_end)]

# --- Compute county averages for each week ---
this_week_avg = this_week.groupby("county", as_index=False).agg(avg_price=("retail", "mean"))
last_week_avg = last_week.groupby("county", as_index=False).agg(avg_price=("retail", "mean"))

# --- Merge and calculate percent change ---
weekly_change = (
    pd.merge(this_week_avg, last_week_avg, on="county", suffixes=("_this_week", "_last_week"))
    .assign(pct_change=lambda x: ((x["avg_price_this_week"] - x["avg_price_last_week"]) / x["avg_price_last_week"]) * 100)
    .sort_values("pct_change", ascending=True)
    .round(2)
)

print(f"Weekly County Price Change ({last_week_start.date()} → {end_of_week.date()})")
weekly_change

Weekly County Price Change (2025-10-06 → 2025-10-19)


,county,avg_price_this_week,avg_price_last_week,pct_change
0,Kisumu,50.0,63.75,-21.57
1,Meru,55.0,53.33,3.12
2,Nairobi,80.0,75.62,5.79
3,Trans-Nzoia,57.5,53.00,8.49


##### **Open AI**

In [ ]:
# --- Initialize client with your key (use Kaggle secret ideally) ---
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# --- Prepare data summaries ---
national_summary = national_weekly.tail(2).to_string(index=False)
county_summary = weekly_change.head(10).to_string(index=False)

# --- Build prompt ---
prompt = f"""
 Find me a unique insight for this week... look for something out 
 of the ordinary: an anomaly, a trend breakout, or something simply 
 interesting I could share.
 1. What happened? (Lead sentence)
 2. By how much? (Key numbers)
 3. So what? (Why it matters)
 4. Why did it happen? (Possible reasons)
 5. What does it mean going forward? (Implications)
 6. Closing line (wrap-up)
 Avoid using markdown formats when generating insights, stick to normal test


1. National Weekly Price Change
{national_summary}

2. County-Level Weekly Price Change:
{county_summary}
"""

# --- Generate summary using OpenAI ---
response = client.responses.create(
    model="gpt-4o-mini",  # use lightweight, fast model
    input=prompt,
    store=True
)

# --- Display AI-generated insight ---
print(response.output_text)

This week, we’re seeing a notable shift in dry maize prices that might have you feeling the pinch at the grocery store. The average retail price of maize has risen to Ksh62.14, marking an increase of 7.58% from last week's Ksh57.76. This upward trend can be concerning, as it directly affects how much you spend on staple foods for your family. With households trying to balance already tight budgets, this increase may lead you to reconsider your shopping habits or meal planning for the week ahead.

Looking closer at regional prices, some counties are experiencing significant fluctuations. For instance, Kisumu has seen a sharp decrease in maize prices to Ksh50, a drop of over 21%, while Nairobi's prices have gone up to Ksh80, reflecting an increase of 5.79%. In Trans-Nzoia and Meru, prices have also risen but are still manageable, sitting at Ksh57.5 and Ksh55 respectively. This disparity means that while you might have to dig a little deeper for maize in some areas, there are savings to b

In [43]:
# --- Your Gmail credentials ---
sender_email = "adrianjuliusaluoch@gmail.com"
app_password = "hgsp doxa czgw mlly" 
# app_password = 'rnki zhzk dhxe mfoc'

# --- Recipients ---
# Recipient list
recipients = [
    "adrianjulius04@gmail.com",
    "adrianaluoch5@gmail.com",
    "adrianaluoch12@gmail.com"
]

In [46]:
# Loop through each recipient
for email in recipients:
    msg = MIMEMultipart("alternative")
    msg["From"] = f"Adrian Julius Aluoch <{sender_email}>"
    msg["To"] = email
    msg["Subject"] = "Weekly Maize Market Brief"

    # Personalized email body
    body = textwrap.dedent(f"""
Hi ,
        
{response.output_text}
        
Best regards,  
Adrian Julius Aluoch
        """)

    msg.attach(MIMEText(body, "plain"))

    # Send email
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender_email, app_password)
        server.send_message(msg)

    # Add Timer
    time.sleep(2)  # Wait 2 seconds before next email

    # Print Message
    print(f"✅ Email sent successfully to ({email})")

✅ Email sent successfully to (adrianjulius04@gmail.com)
✅ Email sent successfully to (adrianaluoch5@gmail.com)
✅ Email sent successfully to (adrianaluoch12@gmail.com)
